# Python Bitcoin notification app
https://realpython.com/python-bitcoin-ifttt/
### Thoughts
Bad tutorial, code examples impletmented are not pythonic

## Storing API keys securely
https://www.google.com/search?client=firefox-b-d&q=storing+api+keys+in+python+projects+best+practices#kpvalbx=_ct3yYrCRKNTzgAakq5LwDw11
1. Install the following library: `pip install python-dotenv`
2. Create a `.env` file in your working directory (add this to your .gitignore file)
    * Add the APIKey to the file as a variable ('' not needed)
3. Import library: `from dotenv import load_env`
4. Create a configure function that calls `load_dotenv`
5. To access the .env variables import the os library
6. To access the variable use `os.getenv("name of .env variable")

In [12]:
from requests import Request, Session
import requests
from dotenv import load_dotenv
import os
import json
import time
from datetime import datetime

In [13]:
def configure():
    load_dotenv()

In [17]:
# IFTTT webhook test
ifttt_webhook_url = f'https://maker.ifttt.com/trigger/test_event/with/key/{os.getenv("IFTTT_KEY")}'
requests.post(ifttt_webhook_url)

<Response [200]>

In [41]:
BITCOIN_PRICE_THRESHOLD=20000

def coin_data():
    '''
    Function returns all data from url in json format
    '''
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
    'start':'1',
    'limit':'5000',
    'convert':'GBP'
    }
    headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': os.getenv('API_KEY'),
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params= parameters)
        data = json.loads(response.text)
        # print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        
    return round(float(data['data']['name'=='bitcoin']['quote']['GBP']['price']),2)

def post_ifttt_webhook(event, value):
    '''
        sends a post request to IFTTT: event must trigger an event name in IFTTT
    '''
    # payload to be sent to IFTTT service
    data = {'value1':value} #key value must agree to var name set up in ifttt
    ifttt_webhook_url = f'https://maker.ifttt.com/trigger/{{}}/with/key/{os.getenv("IFTTT_KEY")}'
    ifttt_event_url = ifttt_webhook_url.format(event)
    # sent a post request to webhook url
    requests.post(ifttt_event_url, json= data)

def format_bitcoin_history(bitcoin_history):
    '''
        takes a list as input and formats it to http
    '''
    rows = []
    for bitcoin_price in bitcoin_history:
        date = bitcoin_price['date'].strftime('%d.%m.%Y %H:%M')
        price = bitcoin_price['price']
        row = '{}: $<b>{}</b>'.format(date, price)
        rows.append(row)
        return '<br>'.join(rows)

def main():
    bitcoin_history=[]
    while True:
        price = coin_data() #gets latest btc price
        date = datetime.now()
        bitcoin_history.append({'date': date, 'price': price}) # appends date price dict to list

        #Sends an emergency notification
        if float(price) < BITCOIN_PRICE_THRESHOLD:
            post_ifttt_webhook('bitcoin_price_emergency', price)
        
        print(bitcoin_history)
        # Send a telegram notification
        # Once there are 5 items in the price history send an update
        if len(bitcoin_history) == 5:
            post_ifttt_webhook('bitcoin_price_update',
                                format_bitcoin_history(bitcoin_history))
            bitcoin_history = [] # resets history
            return
        
        # sleeps script
        time.sleep(1)

if __name__ == '__main__': # explanation: https://stackoverflow.com/questions/419163/what-does-if-name-main-do
    main()


[{'date': datetime.datetime(2022, 8, 10, 23, 20, 38, 364589), 'price': 19607.25}]
[{'date': datetime.datetime(2022, 8, 10, 23, 20, 38, 364589), 'price': 19607.25}, {'date': datetime.datetime(2022, 8, 10, 23, 20, 40, 465216), 'price': 19607.25}]
[{'date': datetime.datetime(2022, 8, 10, 23, 20, 38, 364589), 'price': 19607.25}, {'date': datetime.datetime(2022, 8, 10, 23, 20, 40, 465216), 'price': 19607.25}, {'date': datetime.datetime(2022, 8, 10, 23, 20, 42, 727491), 'price': 19607.25}]
[{'date': datetime.datetime(2022, 8, 10, 23, 20, 38, 364589), 'price': 19607.25}, {'date': datetime.datetime(2022, 8, 10, 23, 20, 40, 465216), 'price': 19607.25}, {'date': datetime.datetime(2022, 8, 10, 23, 20, 42, 727491), 'price': 19607.25}, {'date': datetime.datetime(2022, 8, 10, 23, 20, 44, 719897), 'price': 19607.25}]
[{'date': datetime.datetime(2022, 8, 10, 23, 20, 38, 364589), 'price': 19607.25}, {'date': datetime.datetime(2022, 8, 10, 23, 20, 40, 465216), 'price': 19607.25}, {'date': datetime.datet